In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset_parser import (
    get_ordered_syllable_for_song,
    get_recording_time_from_filename,
    load_single_bird_syllable_csv,
    split_dataset_by_surgery_date
)

In [3]:
import pprint
file_path = 'Modeling_phys_canary/USA5288_decoded.csv'

results = load_single_bird_syllable_csv(file_path)
print(len(results))
pprint.pprint(results[1])

52151
{'animal_id': 'USA5288',
 'file_name': 'USA5288_45355.32438532_3_4_9_0_38.wav',
 'ordered_and_timed_syllables': [('8', 0.0, 1813.3333333333335),
                                 ('21', 1813.3333333333335, 3348.730158730159),
                                 ('22', 3348.730158730159, 4425.396825396826),
                                 ('21', 4425.396825396826, 4611.587301587302),
                                 ('22', 4611.587301587302, 4619.68253968254),
                                 ('21', 4619.68253968254, 4622.380952380952),
                                 ('22', 4622.380952380952, 4625.079365079366),
                                 ('21', 4625.079365079366, 4630.476190476191),
                                 ('26', 4630.476190476191, 4751.904761904762),
                                 ('23', 4751.904761904762, 5010.952380952382),
                                 ('2', 5010.952380952382, 5785.396825396826),
                                 ('3', 5785.396825396826, 804

In [4]:
import json
from datetime import datetime
json_file_path = 'Modeling_phys_canary/USA5288_creation_data.json'

with open(json_file_path, 'r') as f:
    data = json.load(f)

    surgery_treatment_date = datetime.strptime(
        data['treatment_date'], '%Y-%m-%d')

surgery_treatment_date

datetime.datetime(2024, 4, 9, 0, 0)

In [5]:
results_pre_surgery, results_post_surgery = split_dataset_by_surgery_date(results, surgery_treatment_date)
len(results_pre_surgery), len(results_post_surgery)

(33374, 18777)

In [17]:
single_syllable = []
for result in results_pre_surgery:
    if len(result['ordered_and_timed_syllables']) == 0:
        continue

    single_syllable.append([
        str(s[0]) for s in result['ordered_and_timed_syllables']
    ])

single_syllable[0:1]

[['8',
  '21',
  '22',
  '21',
  '22',
  '21',
  '22',
  '21',
  '26',
  '23',
  '2',
  '3',
  '26',
  '5',
  '11',
  '14']]

In [19]:
#### BUILD YOUR ALPHABET HERE and SYLLABLE MAPPING HERE####
import numpy as np
from pypst.transition_mat import build_alphabet_from_dataset

dataset = single_syllable

alphabet = build_alphabet_from_dataset(dataset)
len(alphabet)

30

In [20]:
from pypst.transition_mat import build_transition_matrix
from pypst.pst_learn import pst_learn

transition_matrix = build_transition_matrix(
    dataset,
    2,
    alphabet=alphabet)


L = 2;
p_min = 0.0073 #1e-15 #.01 #0.0073 #0.1e-15 #0.0073;
g_min = .01 # 1e-15 #0.185;
r = 1.6;
alpha = 17.5;



pst = pst_learn(
    transition_matrix['occurrence_mats'],
    alphabet=transition_matrix['alphabet'],
    N=transition_matrix['N'],
    L=L,
    p_min=p_min,
    g_min=g_min,
    r=r,
    alpha=alpha)